In [1]:
import os
import json
import glob

def extract_reflections():
    # Define the path to the memory files
    memory_dir = './memory'
    
    # Initialize a dictionary to store reflections by model and game
    reflections = {
        "gpt4o": {},
        "o3": {}
    }
    
    # Get all memory files in the directory
    memory_files = glob.glob(os.path.join(memory_dir, "*.json"))
    
    for file_path in memory_files:
        # Extract the file name without extension
        file_name = os.path.basename(file_path)
        
        # Skip files that don't match our expected pattern
        if not (file_name.startswith('gpt4o_') or file_name.startswith('o3_')):
            continue
        
        # Determine the model and game from the file name
        parts = file_name.split('_')
        model = parts[0]  # 'gpt4o' or 'o3'
        game = parts[1].split('.')[0]  # 'tetris', 'candy', 'super', '2048'
        
        # Read the memory file
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                memory_data = json.load(f)
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            continue
        
        # Extract only reflections
        game_reflections = []
        for entry in memory_data:
            if "reflection" in entry and entry["reflection"]:
                # Add only the reflection text
                game_reflections.append(entry["reflection"])
        
        # Store reflections for this game in the appropriate model's dictionary
        reflections[model][game] = game_reflections
    
    # Write the extracted reflections to a new JSON file
    output_file = 'all_reflections.json'
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(reflections, f, indent=2)
    
    print(f"Reflections extracted and saved to {output_file}")

if __name__ == "__main__":
    extract_reflections()

Reflections extracted and saved to all_reflections.json


In [2]:
import os
import json
import glob
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from pathlib import Path

def extract_game_data():
    # Define the path to the game log files
    game_log_dir = './game_logs'
    
    # Initialize a dictionary to store data by model
    game_data = {
        "gpt4o": [],
        "o3": []
    }
    
    # Load reflections from the existing file
    try:
        with open('all_reflections.json', 'r', encoding='utf-8') as f:
            all_reflections = json.load(f)
    except Exception as e:
        print(f"Error loading reflections: {e}")
        all_reflections = {"gpt4o": {"2048": []}, "o3": {"2048": []}}
    
    # Get all 2048 game log files
    game_log_files = glob.glob(os.path.join(game_log_dir, "*_2048_game_log.jsonl"))
    
    for file_path in game_log_files:
        # Extract the model from the file name
        file_name = os.path.basename(file_path)
        model = file_name.split('_')[0]  # 'gpt4o' or 'o3'
        
        if model not in game_data:
            continue
            
        # Read the game log file
        try:
            entries = []
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    try:
                        entry = json.loads(line.strip())
                        entries.append(entry)
                    except json.JSONDecodeError:
                        continue
            
            # Get only the last 10 entries
            last_ten_entries = entries[-10:] if len(entries) >= 10 else entries
            
            # Get reflections for this model
            model_reflections = all_reflections.get(model, {}).get("2048", [])
            reflection_count = len(model_reflections)
            
            # Extract required elements
            for i, entry in enumerate(last_ten_entries):
                extracted_data = {}
                if "move" in entry:
                    extracted_data["move"] = entry["move"]
                if "thought" in entry:
                    extracted_data["thought"] = entry["thought"]
                if "perception_data" in entry and "board" in entry["perception_data"]:
                    extracted_data["board"] = entry["perception_data"]["board"]
                    
                    # Create a board image for visualization
                    if "board" in extracted_data:
                        # Create directory for images if it doesn't exist
                        output_dir = os.path.join('board_images', model)
                        os.makedirs(output_dir, exist_ok=True)
                        
                        # Generate a unique filename based on the index
                        index = len(game_data[model])
                        image_path = os.path.join(output_dir, f"board_{index}.png")
                        
                        # Create and save the board image
                        create_board_image(extracted_data["board"], image_path)
                        extracted_data["image_path"] = image_path
                
                # Add reflection if available (cycling through available reflections)
                if reflection_count > 0:
                    reflection_index = i % reflection_count
                    extracted_data["reflection"] = model_reflections[reflection_index]
                
                game_data[model].append(extracted_data)
                
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            continue
    
    # Write the extracted data to a new JSON file
    output_file = 'game_data.json'
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(game_data, f, indent=2)
    
    print(f"Game data extracted and saved to {output_file}")
    print(f"Board images saved to board_images/")

def create_board_image(board, save_path, size=400):
    """Create a visualization of the 2048 board."""
    cell_size = size // 4
    padding = cell_size // 10
    
    # Create a new image with a beige background
    img = Image.new('RGB', (size, size), (250, 248, 239))
    draw = ImageDraw.Draw(img)
    
    # Color mapping for different tile values
    colors = {
        0: (205, 193, 180),      # Empty cell
        2: (238, 228, 218),      # 2
        4: (237, 224, 200),      # 4
        8: (242, 177, 121),      # 8
        16: (245, 149, 99),      # 16
        32: (246, 124, 95),      # 32
        64: (246, 94, 59),       # 64
        128: (237, 207, 114),    # 128
        256: (237, 204, 97),     # 256
        512: (237, 200, 80),     # 512
        1024: (237, 197, 63),    # 1024
        2048: (237, 194, 46),    # 2048
        4096: (60, 58, 50),      # 4096
        8192: (60, 58, 50)       # 8192
    }
    
    # Text colors
    dark_text = (119, 110, 101)  # For small values (2, 4)
    light_text = (249, 246, 242) # For large values (8+)
    
    try:
        # Try to load a better font, fall back to default if not available
        try:
            # Font size base - will be adjusted for larger numbers
            base_font_size = cell_size // 3
            
            # Try common font paths
            potential_fonts = [
                "arial.ttf",
                "Arial.ttf",
                "DejaVuSans-Bold.ttf",
                "LiberationSans-Bold.ttf",
                "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf",
                "/System/Library/Fonts/SFNSDisplay-Bold.otf",  # macOS
                "C:/Windows/Fonts/Arial.ttf",  # Windows
                "C:/Windows/Fonts/ArialBd.ttf",  # Windows Bold
            ]
            
            font = None
            for font_name in potential_fonts:
                try:
                    font = ImageFont.truetype(font_name, base_font_size)
                    break
                except (OSError, IOError):
                    continue
                    
            # Fall back to default if no font found
            if font is None:
                font = ImageFont.load_default()
        except Exception:
            font = ImageFont.load_default()
        
        # Draw the background grid
        draw.rectangle([0, 0, size, size], fill=(187, 173, 160))
        
        # Draw each cell
        for row in range(4):
            for col in range(4):
                # Get power value and convert to actual 2048 value
                power = int(board[row][col])
                value = 0 if power == 0 else 2**power
                
                # Calculate position
                x0 = col * cell_size + padding
                y0 = row * cell_size + padding
                x1 = (col + 1) * cell_size - padding
                y1 = (row + 1) * cell_size - padding
                
                # Draw cell background
                cell_color = colors.get(value, (60, 58, 50))  # Default to dark color for large values
                draw.rectangle([x0, y0, x1, y1], fill=cell_color)
                
                # Skip text for empty cells
                if value == 0:
                    continue
                
                # Choose text color based on value
                text_color = light_text if value > 4 else dark_text
                
                # Draw the value text
                text = str(value)
                
                # Adjust font size based on number length
                font_size = base_font_size
                if len(text) == 3:
                    font_size = int(base_font_size * 0.8)
                elif len(text) >= 4:
                    font_size = int(base_font_size * 0.65)
                
                # Get font with correct size
                adjusted_font = None
                for font_name in potential_fonts:
                    try:
                        adjusted_font = ImageFont.truetype(font_name, font_size)
                        break
                    except (OSError, IOError):
                        continue
                
                if adjusted_font:
                    font = adjusted_font
                
                # Get text size
                if hasattr(font, 'getbbox'):
                    # For newer PIL versions
                    bbox = font.getbbox(text)
                    text_width = bbox[2] - bbox[0]
                    text_height = bbox[3] - bbox[1]
                elif hasattr(font, 'getsize'):
                    # For older PIL versions
                    text_width, text_height = font.getsize(text)
                else:
                    # Fallback estimation
                    text_width = len(text) * font_size // 2
                    text_height = font_size
                
                # Calculate center of cell
                cell_center_x = (x0 + x1) // 2
                cell_center_y = (y0 + y1) // 2
                
                # Calculate text position for perfect centering
                text_x = cell_center_x - text_width // 2
                text_y = cell_center_y - text_height // 2 - (cell_size // 15)
                
                # Draw the text
                draw.text((text_x, text_y), text, fill=text_color, font=font)
                
                # For larger numbers, draw the text slightly bolder
                if value >= 8:
                    draw.text((text_x+1, text_y), text, fill=text_color, font=font)
        
        # Ensure the directory exists
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        
        # Save the image
        img.save(save_path)
        
    except Exception as e:
        print(f"Error creating board image: {e}")

if __name__ == "__main__":
    extract_game_data()

Game data extracted and saved to game_data.json
Board images saved to board_images/
